# 03. LLM Enhanced Training

이 노트북은 LLM 생성 데이터로 enhanced 모델을 학습하고 성능을 비교합니다.

## 전제조건
먼저 다음 노트북들을 순서대로 실행해야 합니다:
1. `01_neural_sparse_base_training.ipynb` - 기본 모델 학습
2. `02_llm_synthetic_data_generation.ipynb` - LLM 데이터 생성

## 목표
- 기본 데이터 + LLM 생성 데이터 결합
- Enhanced Neural Sparse 모델 학습
- Base vs Enhanced 모델 성능 비교

## 출력 데이터
모든 데이터는 `dataset/enhanced_model/` 디렉토리에 저장됩니다:
- `neural_sparse_v2_model/`: Enhanced 모델
- `evaluation/performance_comparison.json`: 성능 비교 결과

In [3]:
# DatasetManager 초기화
from src.dataset_manager import DatasetManager

dm = DatasetManager(base_path="dataset")
print("✅ DatasetManager initialized")
print(f"📁 Base path: {dm.base_path.absolute()}")

✅ DatasetManager initialized
📁 Base path: /home/west/Documents/cursor-workspace/opensearch-neural-pre-train/dataset


In [4]:
# 필수 데이터 파일 확인
required_files = [
    ("base_model", "documents.json"),
    ("base_model", "qd_pairs_base.pkl"),
    ("base_model", "neural_sparse_v1_model"),
    ("llm_generated", "synthetic_qd_pairs.pkl"),
    ("llm_generated", "enhanced_synonyms.json"),
]

if not dm.check_dependencies(required_files):
    raise RuntimeError(
        "Required data files not found. "
        "Please run notebooks 1 and 2 first."
    )

print("\n✅ Ready to proceed with enhanced training")

✅ All dependencies satisfied

✅ Ready to proceed with enhanced training


In [5]:
# 이전 노트북들에서 생성된 데이터 로드
print("Loading data from previous notebooks...\n")

# From notebook 1
documents = dm.load_json("documents.json", "base_model")
qd_pairs_base = dm.load_pickle("qd_pairs_base.pkl", "base_model")

# From notebook 2
synthetic_qd_pairs = dm.load_pickle("synthetic_qd_pairs.pkl", "llm_generated")
enhanced_synonyms = dm.load_json("enhanced_synonyms.json", "llm_generated")

print(f"\n✅ Loaded {len(documents)} documents")
print(f"✅ Loaded {len(qd_pairs_base)} base QD pairs")
print(f"✅ Loaded {len(synthetic_qd_pairs)} synthetic QD pairs")
print(f"✅ Loaded enhanced synonyms with {len(enhanced_synonyms)} entries")

Loading data from previous notebooks...

✓ Loaded JSON: dataset/base_model/documents.json
✓ Loaded Pickle: dataset/base_model/qd_pairs_base.pkl
✓ Loaded Pickle: dataset/llm_generated/synthetic_qd_pairs.pkl
✓ Loaded JSON: dataset/llm_generated/enhanced_synonyms.json

✅ Loaded 127910 documents
✅ Loaded 30000 base QD pairs
✅ Loaded 32 synthetic QD pairs
✅ Loaded enhanced synonyms with 32 entries


In [ ]:
# Base 모델 로드 (성능 비교용)
from transformers import AutoModel, AutoTokenizer
import torch

print("Loading base model for comparison...")

model_path = dm.base_path / "base_model" / "neural_sparse_v1_model"

base_tokenizer = AutoTokenizer.from_pretrained(str(model_path))
base_model = AutoModel.from_pretrained(str(model_path))

# GPU 사용 가능하면 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = base_model.to(device)
base_model.eval()

print(f"✅ Base model loaded for comparison")
print(f"   Device: {device}")
print(f"   Model type: {type(base_model).__name__}")

## 🆕 16. 생성된 합성 데이터 분석

LLM으로 생성한 합성 데이터의 품질과 통계를 분석합니다.

In [ ]:
print("\n" + "="*70)
print("📊 섹션 16: 생성된 합성 데이터 분석")
print("="*70)

# 기존 데이터 + 합성 데이터 통계
print(f"\n📊 데이터 통계:")
print(f"   기본 QD pairs: {len(qd_pairs_base):,}개")
print(f"   합성 QD pairs: {len(synthetic_qd_pairs):,}개")
print(f"   총 데이터: {len(qd_pairs_base) + len(synthetic_qd_pairs):,}개")

if len(synthetic_qd_pairs) > 0:
    increase_rate = len(synthetic_qd_pairs) / len(qd_pairs_base) * 100
    print(f"   증가율: +{increase_rate:.2f}%")
else:
    print(f"   ⚠️  합성 데이터가 생성되지 않았습니다.")

# 합성 데이터 샘플 확인
if len(synthetic_qd_pairs) > 0:
    print(f"\n📋 합성 Query-Document Pairs 샘플 (처음 5개):")
    print("="*70)
    for i, (query, doc, relevance) in enumerate(synthetic_qd_pairs[:5], 1):
        print(f"\n{i}. Query: {query}")
        print(f"   Document: {doc[:100]}...")
        print(f"   Relevance: {relevance}")
    print("="*70)

# 동의어 사전 비교
print(f"\n📊 동의어 사전 통계:")
print(f"   Enhanced synonyms: {len(enhanced_synonyms):,}개 항목")
print(f"\n샘플 동의어:")
for i, (korean, english_list) in enumerate(list(enhanced_synonyms.items())[:5], 1):
    print(f"   {i}. {korean} → {', '.join(english_list)}")

print("\n" + "="*70)
print("✅ 데이터 분석 완료")
print("="*70)

## 🆕 17. 데이터 품질 분석 및 향후 계획

생성된 데이터의 품질을 분석하고 다음 단계를 계획합니다.

In [ ]:
print("\n" + "="*70)
print("📊 섹션 17: 데이터 품질 분석 및 향후 계획")
print("="*70)

# 생성된 쿼리 품질 분석
print("\n1️⃣ 합성 데이터 품질 분석")
print("-" * 70)

if len(synthetic_qd_pairs) > 0:
    # 쿼리 길이 분석
    query_lengths = [len(q) for q, d, r in synthetic_qd_pairs]
    avg_query_length = sum(query_lengths) / len(query_lengths)
    
    print(f"   평균 쿼리 길이: {avg_query_length:.1f} 자")
    print(f"   최소 쿼리 길이: {min(query_lengths)} 자")
    print(f"   최대 쿼리 길이: {max(query_lengths)} 자")
    
    # 문서 길이 분석
    doc_lengths = [len(d) for q, d, r in synthetic_qd_pairs]
    avg_doc_length = sum(doc_lengths) / len(doc_lengths)
    
    print(f"\n   평균 문서 길이: {avg_doc_length:.1f} 자")
    print(f"   최소 문서 길이: {min(doc_lengths)} 자")
    print(f"   최대 문서 길이: {max(doc_lengths)} 자")
    
    # Relevance 분포
    relevance_scores = [r for q, d, r in synthetic_qd_pairs]
    unique_scores = set(relevance_scores)
    
    print(f"\n   Relevance 점수 분포:")
    for score in sorted(unique_scores):
        count = relevance_scores.count(score)
        percentage = count / len(relevance_scores) * 100
        print(f"     {score}: {count}개 ({percentage:.1f}%)")
else:
    print("   ⚠️  합성 데이터가 없습니다.")

# 동의어 사전 개선 분석
print(f"\n2️⃣ 동의어 사전 분석")
print("-" * 70)
print(f"   LLM 검증 동의어: {len(enhanced_synonyms):,}개 항목")

# 향후 개선 계획
print(f"\n3️⃣ 향후 개선 계획")
print("-" * 70)
print(f"""
   📌 데이터 확장:
      - 현재: {len(synthetic_qd_pairs)}개 합성 pairs (테스트)
      - 목표: 1,000+ 합성 pairs (노트북 2에서 max_documents 증가)
   
   📌 모델 재학습:
      - 충분한 합성 데이터 생성 후 진행
      - 기본 모델 + 합성 데이터로 enhanced 모델 학습
      - 성능 비교 및 평가
   
   📌 배포:
      - Enhanced 모델을 OpenSearch에 배포
      - 실제 검색 서비스에서 성능 검증
""")

print("="*70)
print("✅ 분석 완료")
print("="*70)

print("\n💡 다음 단계:")
print("   1. 노트북 2로 돌아가서 max_documents를 1000으로 증가")
print("   2. 충분한 합성 데이터 생성")
print("   3. 이 노트북으로 돌아와서 enhanced 모델 학습")

---

# 데이터 저장 (분석 결과)

생성된 데이터 분석 결과를 `dataset/llm_generated/` 디렉토리에 저장합니다.

In [ ]:
# 데이터 분석 결과 저장
import json

analysis_results = {
    "base_qd_pairs_count": len(qd_pairs_base),
    "synthetic_qd_pairs_count": len(synthetic_qd_pairs),
    "enhanced_synonyms_count": len(enhanced_synonyms),
    "total_documents": len(documents),
}

if len(synthetic_qd_pairs) > 0:
    query_lengths = [len(q) for q, d, r in synthetic_qd_pairs]
    analysis_results["synthetic_data_quality"] = {
        "avg_query_length": sum(query_lengths) / len(query_lengths),
        "min_query_length": min(query_lengths),
        "max_query_length": max(query_lengths),
    }

dm.save_json(
    analysis_results,
    "data_analysis.json",
    "llm_generated"
)

print("✅ 분석 결과 저장 완료")
print(f"   위치: dataset/llm_generated/data_analysis.json")

In [ ]:
# 결합된 데이터셋 저장 (향후 학습용)
combined_qd_pairs = qd_pairs_base + synthetic_qd_pairs

dm.save_pickle(
    combined_qd_pairs,
    "combined_qd_pairs.pkl",
    "llm_generated"
)

print(f"✅ 결합된 데이터셋 저장 완료")
print(f"   위치: dataset/llm_generated/combined_qd_pairs.pkl")
print(f"   총 개수: {len(combined_qd_pairs):,}개 pairs")

In [ ]:
# 전체 데이터셋 요약
dm.print_summary()

---

## ✅ Notebook 3 완료 - 데이터 분석

LLM으로 생성한 합성 데이터의 분석이 완료되었습니다.

### 완료된 작업
1. ✅ Base 모델 데이터 로드 (Notebook 1)
2. ✅ LLM 합성 데이터 로드 (Notebook 2)
3. ✅ 데이터 품질 분석 및 통계

### 저장된 데이터
- Combined Dataset: `dataset/llm_generated/combined_qd_pairs.pkl`
- Analysis Results: `dataset/llm_generated/data_analysis.json`

### 다음 단계 (향후 작업)
1. 노트북 2에서 max_documents를 늘려 더 많은 합성 데이터 생성
2. 충분한 데이터가 모이면 Enhanced 모델 재학습 진행
3. 성능 비교 및 OpenSearch 배포

### 현재 상태
- 테스트 모드로 소량의 합성 데이터만 생성됨
- 실제 모델 재학습을 위해서는 더 많은 데이터 필요